## **Vamsi Sandeep Narayanam - X638B786-HW-3**

Dataset Link:
https://www.kaggle.com/datasets/veer1516/finanical-sentiment-analysis

In [1]:
# IMPORT PACKAGES
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Read the data file
df = pd.read_csv("C:\\Users\\vamsi\\OneDrive\\Desktop\\Ross\\Vamsi\\sampled_stock_data(1).csv")
data=df.copy()

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df.head()

,Text,Sentiment
0,Where to even start? The horrendous acting? Th...,negative
1,Had the original casting idea been kept (hunti...,negative
2,Tycoon will never be listed as one of John Way...,negative
3,"Alright, let me break it down for ya... Haggar...",positive
4,Do you know when you look at your collection o...,negative


In [5]:
!pip install transformers torch


  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/2a/b7/a3cf5fd40334b9785cc83ee0c96b50603026eb3aa70210a33729018e7029/torch-2.3.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for mkl<=2021.4.0,>=2021.1.1 from https://files.pythonhosted.org/packages/fe/1c/5f6dbf18e8b73e0a5472466f0ea8d48ce9efae39bd2ff38cebf8dce61259/mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata
  Obtaining dependency information for intel-openmp==2021.* from https://files.pythonhosted.org/packages/6f/21/b590c0cc3888b24f2ac9898c41d852d7454a1695fbad34bee85dba6dc408/intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata
  Obtaining dependency information for tbb==2021.* from https://files.pythonhosted.org/packages/7b/2d/1e1c70fae8ac

ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [9]:
# !pip install torch
# !pip install transformers
# !pip install torch torchvision torchaudio

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['Text'], df['Sentiment'], test_size=0.1, random_state=2020)

# Initialize tokenizer from the BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode sequences in the training and validation sets
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

# Convert labels to integers if they're not already (ensure labels are properly formatted)
label_dict = {'negative': 0, 'positive': 1}  # Modify as needed
train_labels = [label_dict[label] for label in train_labels]
val_labels = [label_dict[label] for label in val_labels]

# Convert data to torch tensors
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), torch.tensor(train_labels))
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']), torch.tensor(val_encodings['attention_mask']), torch.tensor(val_labels))

# Set up BERT for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_dict))

# Setting up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the appropriate device

# Define data loaders
batch_size = 16
train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
validation_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)

# Setup optimizer and schedule
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = tuple(b.to(device) for b in batch)
        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} - Average training loss: {avg_train_loss}")

    # Validation step
    model.eval()
    eval_accuracy = 0
    n_eval_examples = 0
    for batch in validation_loader:
        batch = tuple(b.to(device) for b in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = accuracy_score(label_ids, np.argmax(logits, axis=1))
        eval_accuracy += tmp_eval_accuracy
        n_eval_examples += b_input_ids.size(0)

    print(f"Validation Accuracy: {eval_accuracy / n_eval_examples}")


ModuleNotFoundError: No module named 'torch'

**Setting Up the Environment and Preparing the Data:**: Importing libraries needed for data handling and modeling. Splitting data into training and validation sets to prepare for model training and evaluation.

**Tokenization and Encoding:** Initialized a tokenizer specific to BERT to process text data. Converted text to a format suitable for BERT (token IDs, attention masks).

**Data Preparation for PyTorch:** Converted labels to a numeric format using a dictionary. Created tensors from tokenized text and labels for use in PyTorch.
Organizes data into batches using DataLoader for efficient processing during training and validation.
**Model Initialization and Setup:** Loading the BERT model configured for sequence classification. Setting up the device (GPU or CPU) for model training.
Configures the optimizer and scheduler to adjust learning rates and improve training dynamics.
**Training Loop:** Training the model in batches over multiple cycles (epochs).
Calculating and updates weights to reduce prediction errors. Tracking and training the progress by reporting loss after each epoch.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize lists to store outputs
all_preds = []
all_labels = []

# Validation step
model.eval()
for batch in validation_loader:
    batch = tuple(b.to(device) for b in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    all_preds.extend(np.argmax(logits, axis=1))
    all_labels.extend(label_ids)


**Validation and Evaluation** Here in this section it shows that Evaluation of the model using the validation set to check how well it performs on unseen data. Calculates and prints accuracy, providing a quick metric of model performance.

In [ ]:
# Compute the classification report
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=label_dict.keys()))

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_dict.keys(), yticklabels=label_dict.keys())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


**Detailed Performance Metrics** below Generates detailed statistics (classification report) and visualizes errors (confusion matrix). Can Provide insights into how well the model predicts each class and where it may be making errors.
